In [1]:
%%capture
#%pip install llama-index openai==1.30.1 llama-index-embeddings-openai llama-index-embeddings-cohere==0.1.9 qdrant-client llama-index-vector-stores-qdrant llama-index-llms-openai llama-index-llms-cohere==0.1.19
%pip install llama-index mistralai llama-index-embeddings-mistralai qdrant-client llama-index-vector-stores-qdrant llama-index-llms-mistralai

In [2]:
import os
import sys
from getpass import getpass
import nest_asyncio

from IPython.display import Markdown, display

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv("")

sys.path.append('../helpers')

from utils import setup_llm, setup_embed_model, setup_vector_store

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/conda/envs/llama/lib/python3.13/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
MISTRAL_API_KEY = os.environ['MISTRAL_API_KEY']

In [4]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [5]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [6]:
from llama_index.core.settings import Settings
from utils import setup_llm, setup_embed_model

setup_llm(
    provider="mistral", 
    model="mistral-small-latest", 
    api_key=MISTRAL_API_KEY, 
    temperature=0.75, 
    system_prompt="""Use ONLY the provided context and generate a complete, coherent answer to the user's query. 
    Your response must be grounded in the provided context and relevant to the essence of the user's query.
    """
    )

setup_embed_model(
    provider="mistral",
    api_key=MISTRAL_API_KEY
    )

In [7]:
from utils import get_documents_from_docstore

senpai_documents = get_documents_from_docstore("../data/words-of-the-senpais")

In [8]:
print(senpai_documents[42].text)

Set a very high hourly aspirational rate for yourself and stick to it. It should seem and feel absurdly high. If it doesnt, its not high enough. Whatever you picked, my advice to you would be to raise it. Like I said, for myself, even before I had money, for the longest time I used $5,000 an hour. And if you extrapolate that out into what it looks like as an annual salary, its multiple millions of dollars per year. Ironically, I actually think Ive beaten it. Im not the hardest working personIm actually a lazy person. I work through bursts of energy where Im really motivated with something. If I actually look at how much Ive earned per actual hour that Ive put in, its probably quite a bit higher than that. Can you expand on your statement, If you secretly despise wealth, it will elude you? If you get into a relative mindset, youre always going to hate people who do better than you, youre always going to be jealous or envious of them. Theyll sense those feelings when you try and do busin

# 🔹→🔷 Small to Big Retrieval ◾️ → ⬛️

The concept of small to big retrieval, also known as recursive retrieval, is a key part of LlamaIndex. And, in order to use this, we need to define how to efficiently retrieve relevant context from an index based on a query. That means defining a recursive retrieval strategy, post processing the nodes once they've been retrieved and synthsizing the responses. 

 1) 🔄 **Recursive Retrieval**

  - **Small Chunks (Child Chunks)**: Initially retrieves smaller, query-specific chunks of data.

  - **Big Chunks (Parent Chunks)**: Follows references to larger, contextual chunks related to the smaller chunks. Retains context within each chunk.

  2) 🛠️ **Node Postprocessing:** Apply transformations, filtering, or re-ranking to the retrieved nodes to enhance data quality and relevance.
  
  3) 📝 **Response Synthesizer:** Use the retrieved text chunks along with the user query to generate a response


## [🪟`SentenceWindowNodeParser`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/node_parser/text/sentence_window.py)

The `SentenceWindowNodeParser` is unique in that it focuses on individual sentences while also capturing the surrounding context.  This is particularly useful for tasks where understanding the broader context of a sentence is useful.

### How it Works

1. **Sentence Splitting:** 

    *   Similar to `SentenceSplitter`, it first divides the document into individual sentences using a sentence tokenizer (defaults to [`PunktSentenceTokenizer`](https://www.nltk.org/api/nltk.tokenize.PunktSentenceTokenizer.html) from the `nltk` library).

2. **Window Creation:**

    *   For each sentence (node), it gathers a "window" of surrounding sentences based on the specified `window_size`. 

    *   This window is stored in the node's metadata under the `window_metadata_key`.

3. **Metadata Management:**

    *   The original sentence text is also stored in the metadata under `original_text_metadata_key`.

    *   Importantly, both the window and original text are excluded from being seen by the embedding model and LLM.

### Arguments you need to know

*   **`window_size`**: Controls the number of sentences to include before and after the central sentence in the window.

*   **`window_metadata_key`**: The key used to store the window text in the node's metadata.

*   **`original_text_metadata_key`**: The key used to store the original sentence text in the metadata.

*   **`sentence_splitter`**: The text splitter to use when splitting documents (defaults to [`PunktSentenceTokenizer`](https://www.nltk.org/api/nltk.tokenize.PunktSentenceTokenizer.html) from the `nltk` library).

### Usage Example

```python
from llama_index.core.node_parser import SentenceWindowNodeParser

parser = SentenceWindowNodeParser(window_size=2)

nodes = parser.get_nodes_from_documents(documents)
```

### When to Use `SentenceWindowNodeParser`

*   **Tasks requiring sentence-level understanding with context:** 
    *   Question answering, summarization, or sentiment analysis where the surrounding sentences provide valuable context.

*   **Fine-grained control over embedding scope:** 
    *   Creating embeddings that focus on the specific meaning of a sentence within its local context.
    
*   **Combining with MetadataReplacementNodePostProcessor:**
    *   Replacing the original sentence with its surrounding window before sending it to the LLM, allowing the model to consider the broader context.


In [9]:
senpai_documents[42].__dict__

{'id_': '0bd19825-4d4f-48e3-a267-c3d0dff337ec',
 'embedding': None,
 'metadata': {'page_number': 42,
  'file_name': '../data/almanack_of_naval_ravikant.pdf',
  'title': 'The Almanack of Naval Ravikant',
  'author': 'Naval Ravikant'},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {},
 'text': 'Set a very high hourly aspirational rate for yourself and stick to it. It should seem and feel absurdly high. If it doesnt, its not high enough. Whatever you picked, my advice to you would be to raise it. Like I said, for myself, even before I had money, for the longest time I used $5,000 an hour. And if you extrapolate that out into what it looks like as an annual salary, its multiple millions of dollars per year. Ironically, I actually think Ive beaten it. Im not the hardest working personIm actually a lazy person. I work through bursts of energy where Im really motivated with something. If I actually look at how much Ive earned per actual hour that Iv

In [10]:
from llama_index.core.node_parser import SentenceWindowNodeParser

example_parsed = SentenceWindowNodeParser(window_size=2).build_window_nodes_from_documents([senpai_documents[42]])

In [11]:
example_parsed[3].__dict__

{'id_': '62ac20b9-4417-4223-bd2d-b5deaf710e09',
 'embedding': None,
 'metadata': {'window': 'It should seem and feel absurdly high.  If it doesnt, its not high enough.  Whatever you picked, my advice to you would be to raise it.  Like I said, for myself, even before I had money, for the longest time I used $5,000 an hour.  And if you extrapolate that out into what it looks like as an annual salary, its multiple millions of dollars per year. ',
  'original_text': 'Whatever you picked, my advice to you would be to raise it. '},
 'excluded_embed_metadata_keys': ['window', 'original_text'],
 'excluded_llm_metadata_keys': ['window', 'original_text'],
 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0bd19825-4d4f-48e3-a267-c3d0dff337ec', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_number': 42, 'file_name': '../data/almanack_of_naval_ravikant.pdf', 'title': 'The Almanack of Naval Ravikant', 'author': 'Naval Ravikant'}, hash='2e2f878162e5f6eed661de1bfce1d9e

In [12]:
example_parsed_2 = SentenceWindowNodeParser(window_size=3).get_nodes_from_documents([senpai_documents[42]])

In [13]:
example_parsed_2[3].__dict__

{'id_': 'd2a3cf83-7dfe-4c37-bf05-466f30e5f103',
 'embedding': None,
 'metadata': {'page_number': 42,
  'file_name': '../data/almanack_of_naval_ravikant.pdf',
  'title': 'The Almanack of Naval Ravikant',
  'author': 'Naval Ravikant',
  'window': 'Set a very high hourly aspirational rate for yourself and stick to it.  It should seem and feel absurdly high.  If it doesnt, its not high enough.  Whatever you picked, my advice to you would be to raise it.  Like I said, for myself, even before I had money, for the longest time I used $5,000 an hour.  And if you extrapolate that out into what it looks like as an annual salary, its multiple millions of dollars per year.  Ironically, I actually think Ive beaten it. ',
  'original_text': 'Whatever you picked, my advice to you would be to raise it. '},
 'excluded_embed_metadata_keys': ['window', 'original_text'],
 'excluded_llm_metadata_keys': ['window', 'original_text'],
 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='

### 🔄 **Understanding the `MetadataReplacementPostProcessor` and `SentenceWindowNodeParser`**

- 📝 **`SentenceWindowNodeParser` Review**

  - **Single Sentence Parsing**: Parses documents into nodes, each containing a single sentence.

  - **Contextual Window**: Each node includes a "window" of sentences surrounding the core sentence for added context.

- 🔄 **[`MetadataReplacementPostProcessor`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/postprocessor/metadata_replacement.py)**

  - **Context Enhancement**: Replaces the sentence in each node with its surrounding window of sentences during retrieval.

  - **Used in Conjunction**: Often paired with the `SentenceWindowNodeParser` to maximize contextual data provided to the LLM (Language Learning Model).

### Query and Response Process

- 🔍 **Query Handling**

  - **Sentence Retrieval**: Retrieves the most relevant sentences based on the query.

  - **Context Injection**: Instead of merely returning these sentences, the post-processor injects the surrounding context from the window.

- 📊 **Benefits of Enhanced Context**

  - **Improved Understanding**: More context helps the LLM understand queries better, leading to more accurate responses.

  - **Detailed Responses**: The additional context allows for responses that are both detailed and relevant.

- 🌟 **Ideal for Large Documents**

  - **Fine-Grained Retrieval**: Especially useful for large documents or indexes, enabling more precise information extraction.

<img src="https://miro.medium.com/v2/resize:fit:2000/0*JKZ9m_c6jyIKqCWu.png">

Image Source: [Ivan Ilin](https://pub.towardsai.net/advanced-rag-techniques-an-illustrated-overview-04d193d8fec6)

In [14]:
from llama_index.core.node_parser import SentenceWindowNodeParser

def sentence_window_splitter(window_size, documents):
    splitter = SentenceWindowNodeParser(
        window_size=window_size,
        window_metadata_key="window_size",
        original_text_metadata_key="original_text",
        )
    nodes = splitter.get_nodes_from_documents(documents)
    return nodes

In [15]:
nodes = sentence_window_splitter(window_size=5, documents=senpai_documents)

In [16]:
nodes[5].__dict__

{'id_': 'f08f52f4-ad24-41c0-a8ba-87d5fb1ca82d',
 'embedding': None,
 'metadata': {'page_number': 0,
  'file_name': '../data/almanack_of_naval_ravikant.pdf',
  'title': 'The Almanack of Naval Ravikant',
  'author': 'Naval Ravikant',
  'window_size': 'UNDERSTAND HOW WEALTH IS CREATED I like to think that if I lost all my money and you dropped me on a random street in any English-speaking country, within five or ten years Id be wealthy again because its just a skillset Ive developed that anyone can develop.  Its not really about hard work.  You can work in a restaurant eighty hours a week, and youre not going to get rich.  Getting rich is about knowing what to do, who to do it with, and when to do it.  It is much more about understanding than purely hard work.  Yes, hard work matters, and you cant skimp on it.  But it has to be directed in the right way.  If you dont know yet what you should work on, the most important thing is to figure it out.  You should not grind at a lot of hard work

In [17]:
print(nodes[5].get_content(metadata_mode="all"))

page_number: 0
file_name: ../data/almanack_of_naval_ravikant.pdf
title: The Almanack of Naval Ravikant
author: Naval Ravikant
window_size: UNDERSTAND HOW WEALTH IS CREATED I like to think that if I lost all my money and you dropped me on a random street in any English-speaking country, within five or ten years Id be wealthy again because its just a skillset Ive developed that anyone can develop.  Its not really about hard work.  You can work in a restaurant eighty hours a week, and youre not going to get rich.  Getting rich is about knowing what to do, who to do it with, and when to do it.  It is much more about understanding than purely hard work.  Yes, hard work matters, and you cant skimp on it.  But it has to be directed in the right way.  If you dont know yet what you should work on, the most important thing is to figure it out.  You should not grind at a lot of hard work until you figure out what you should be working on.  I came up with the principles in my tweetstorm (below) fo

In [18]:
print(nodes[5].get_content(metadata_mode="llm"))

page_number: 0
file_name: ../data/almanack_of_naval_ravikant.pdf
title: The Almanack of Naval Ravikant
author: Naval Ravikant

Yes, hard work matters, and you cant skimp on it.


## 👷🏽‍♂️ 🗂️ Ingest to Qdrant And Build the Index

In [19]:
from llama_index.core import StorageContext
from llama_index.core.settings import Settings

from utils import create_index, create_query_engine
from utils import setup_vector_store

COLLECTION_NAME = "wots-small-to-big-sentence-window"

sentence_window_vector_store = setup_vector_store(QDRANT_URL, QDRANT_API_KEY, COLLECTION_NAME)


Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


In [20]:
from utils import ingest

transforms = [Settings.embed_model]

split_nodes = ingest(
    documents=nodes,
    transformations=transforms,
    vector_store=sentence_window_vector_store
)

### 🛠️ Setup Query Engine

In [22]:
from llama_index.core import PromptTemplate
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

from utils import create_query_engine
from prompts import HYPE_ANSWER_GEN_PROMPT

HYPE_ANSWER_GEN_PROMPT_TEMPLATE = PromptTemplate(HYPE_ANSWER_GEN_PROMPT)

node_postprocessors = [MetadataReplacementPostProcessor(target_metadata_key="window")]

sentence_window_index = create_index(
    from_where="vector_store",
    vector_store=sentence_window_vector_store,
    embed_model=Settings.embed_model,
    )

sentence_window_query_engine = create_query_engine(
    index=sentence_window_index, 
    mode="query",
    response_mode="compact",
    similiarty_top_k=5,
    vector_store_query_mode="mmr", 
    vector_store_kwargs={"mmr_threshold": 0.42},
    node_postprocessors=node_postprocessors
    )

sentence_window_query_engine.update_prompts({'response_synthesizer:text_qa_template':HYPE_ANSWER_GEN_PROMPT_TEMPLATE})

### 🔧 Setup Query Pipeline

In [23]:
from utils import create_query_pipeline

from llama_index.core.query_pipeline import InputComponent

input_component = InputComponent()

sentence_window_chain = [input_component, sentence_window_query_engine]

sentence_window_query_pipeline = create_query_pipeline(sentence_window_chain)

In [24]:
sentence_window_query_pipeline.run(input="How can I effectively build strength across multiple facets of real life without relying on complicated machines?")

> Running module 2813bfd5-6659-4ead-9924-86abc4954102 with input: 
input: How can I effectively build strength across multiple facets of real life without relying on complicated machines?

> Running module 0ef8b507-0272-4d7a-b7fb-cce3b5ac0150 with input: 
input: How can I effectively build strength across multiple facets of real life without relying on complicated machines?



Response(response="Listen up, you've got everything you need to build strength in all areas of life. Nature's given you the tools, now it's time to use them. Concentrate your powers, focus your energy, and don't let anything hold you back. You've got the strength, now unleash it. Find that one exercise, that one practice, that one routine that you can stick to. Go back to it time and again. Make it a habit. Make it a part of you. That's how you build strength. That's how you become unstoppable.", source_nodes=[NodeWithScore(node=TextNode(id_='c9edf171-9786-4d5b-ad38-049fb5a6d981', embedding=None, metadata={'page_number': 191, 'file_name': '../data/taoofseneca_vol3.pdf', 'title': 'Letters From a Stoic Volume 3', 'author': 'Seneca', 'window_size': 'We are mere manikins, unable to deny ourselves everything.  We shall sorrow, but not to any great extent; we shall feel desires, but in moderation; we shall give way to anger, but we shall be appeased.  And do you know why we have not the powe

In [25]:
sentence_window_query_pipeline.run(input="How can I set rules and speak honestly without worrying about hurting someone's feelings?")

> Running module 2813bfd5-6659-4ead-9924-86abc4954102 with input: 
input: How can I set rules and speak honestly without worrying about hurting someone's feelings?

> Running module 0ef8b507-0272-4d7a-b7fb-cce3b5ac0150 with input: 
input: How can I set rules and speak honestly without worrying about hurting someone's feelings?



Response(response='Listen up, you\'re not here to make everyone happy, you\'re here to help. Seneca said it straight up, "Our words should aim not to please, but to help." So, set your rules and speak your truth. Don\'t worry about hurting feelings. If you\'re honest, you\'re helping. People might say, "That\'s just how I feel, that\'s not true or false." But you know what? Feelings aren\'t facts. You\'re doing this to help, not to please. So, be honest, set those rules, and help others grow. This is your time to shine, to be the beacon of truth and honesty.', source_nodes=[NodeWithScore(node=TextNode(id_='59968f21-a76e-4023-be4c-39428b3d847e', embedding=None, metadata={'page_number': 278, 'file_name': '../data/taoofseneca_vol1-1.pdf', 'title': 'Letters From a Stoic Volume 1', 'author': 'Seneca', 'window_size': 'because at some level we believe were about to be harmed.  Thats virtually identical to the Stoic theory of fear, recorded over two thousand years earlier.  This was an importa

# 👨‍👦 Smaller Child Chunks Referring to Bigger Parent Chunk

<img src="https://miro.medium.com/v2/resize:fit:2000/0*x4rMd50GP99OSDuo.png"  width="70%">

Source: [Ivan Ilin](https://pub.towardsai.net/advanced-rag-techniques-an-illustrated-overview-04d193d8fec6)

🔗 **Chunk References Explained:**

- 🧩 **Concept**: Chunk References involve smaller chunks of data pointing to larger parent chunks, forming a hierarchical graph structure.
  
- 🌐 **Purpose**: This method is utilized in recursive retrieval to efficiently manage and access data in a structured manner.

### Process During Query

- 🔍 **During Query-Time**:

  - **Small Chunk Retrieval**: Initially, smaller chunks relevant to the query are retrieved.

  - **Following References**: The system then follows references to retrieve the larger parent chunks associated with these smaller chunks.

- 📈 **Benefits of Contextual Retrieval**:

  - **Enhanced Context**: Retrieving larger chunks along with the smaller ones provides additional context.
  
  - **Improved Responses**: This deeper context allows for more accurate and comprehensive responses to queries.

This structured approach ensures that data retrieval is both efficient and context-rich, enhancing the overall synthesis and response accuracy.

 The code below is creating a system where smaller chunks of text refer to the larger chunks they were created from. This allows for more context to be provided when retrieving chunks of text based on a query.

In [26]:
# Import the SentenceSplitter class from the llama_index.core.node_parser module
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode

# Define the sizes of chunks for sentence splitting
sub_chunk_sizes = [128, 256, 512]

# Create a list of SentenceSplitter instances with different chunk sizes
sub_node_parsers = [SentenceSplitter(chunk_size=c, chunk_overlap=16) for c in sub_chunk_sizes]

# Initialize an empty list to store all index nodes
all_nodes = []

# Iterate over each base node in senpai_documents
for base_node in senpai_documents:
    # Process each base node with every SentenceSplitter in the list
    for n in sub_node_parsers:
        # Get sub-nodes by splitting the base node document into smaller parts
        sub_nodes = n.get_nodes_from_documents([base_node])
        # Convert each sub-node into an IndexNode and link it to the base node's ID
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        # Add the newly created index nodes to the all_nodes list
        all_nodes.extend(sub_inodes)

    # Also add the original base node to the list of all nodes as an IndexNode
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

In [27]:
all_nodes_dict = {n.node_id: n for n in all_nodes}

In [28]:
all_nodes[5].__dict__

{'id_': '33b3ee22-5a2c-4857-8d26-38803665a1b2',
 'embedding': None,
 'metadata': {'page_number': 0,
  'file_name': '../data/almanack_of_naval_ravikant.pdf',
  'title': 'The Almanack of Naval Ravikant',
  'author': 'Naval Ravikant'},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='68e7881a-d87e-4cda-845f-7eb5e88d84c8', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_number': 0, 'file_name': '../data/almanack_of_naval_ravikant.pdf', 'title': 'The Almanack of Naval Ravikant', 'author': 'Naval Ravikant'}, hash='b00d124eb1a97f077d33a4231c4fe920705ea86d2e44ac22c4447d7d6944668a'),
  <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='4c7411b7-d6d9-43b0-ab64-b29c8bb5bcc8', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='5721476dc71bf63412e749cca4609c6bc48ac29d87eef20661c268042c0f3064')},
 'text': 'UNDERSTAND HOW WEALTH IS CREATED I like to think that if I lost all my money and you 

### 👷🏽‍♂️ 🗂️  Ingest to Qdrant and Build the Index 

In [29]:
from utils import ingest

COLLECTION_NAME = "words-of-the-senpai-small-to-big-parent-child"

parent_child_vector_store = setup_vector_store(QDRANT_URL, QDRANT_API_KEY, COLLECTION_NAME)

transforms = [Settings.embed_model]

parent_child_nodes = ingest(
    documents=all_nodes,
    transformations=transforms,
    vector_store=parent_child_vector_store
)

parent_child_index = create_index(
    from_where="vector_store", 
    embed_model=Settings.embed_model,
    vector_store=parent_child_vector_store)

Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


### 🛠️ Setup Query Engine for Parent Child Chunks

We're making use of the `RecursiveRetriever` and the `RetrieverQueryEngine`.

`RecursiveRetriever` is a separate class that is not directly associated with an index. It is uses multiple retrievers and query engines to recursively retrieve and query nodes.

But, we can't directly use a `RecursiveRetriever` with the `index.as_retriever()` pattern we've seen before. The `index.as_retriever()` pattern is used to create a retriever from an index, and the type of retriever it creates depends on the `retriever_mode` argument you pass to it. `RecursiveRetriever` requires a dictionary of retrievers, and optionally a dictionary of query engines and a dictionary of node. There aren't required when creating a retriever using `index.as_retriever()`.

So, we need to build the `RecursiveRetriever` and the `RetrieverQueryEngine` to accomplish this.

  - `RecursiveRetriever` queries a graph of retrievers and query engines, following links between them to fetch relevant information for a given query. It recursively traverses the graph, deduplicates nodes, and returns the retrieved nodes along with any additional source nodes.

  - `RetrieverQueryEngine` is a component that uses a retriever to fetch relevant documents or nodes based on a given query and then *synthesizes a response from the retrieved nodes using a `ResponseSynthesizer`*. It retrieves relevant nodes, applies postprocessing, synthesizes a response, and returns the result.


More details are below.

#### [`RecursiveRetriever`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/retrievers/recursive_retriever.py)

##### 🌿 Initialization

   - Takes a root ID, retriever dict, and optional query engine & node dicts

   - Validates root ID and checks for overlapping keys

##### 🔄 Recursive Retrieval

   - Starts from the root ID when `retrieve` is called with a query bundle

   - Fetches the object (retriever, query engine, or node) for the current ID

   - If it's a node, adds it to the list of nodes to return

   - If it's a retriever, retrieves nodes and recursively queries them

   - If it's a query engine, queries it and adds the response as a text node

##### 🔗 Querying Retrieved Nodes

   - For each retrieved IndexNode, recursively retrieves from the referenced ID

   - For each TextNode, simply adds it to the list of nodes to return

   - Avoids querying the same ID multiple times

##### 🧹 Deduplication

   - Deduplicates nodes based on their node ID

   - Keeps the node with the highest score or the first one returned

##### 📚 Retrieving All Nodes

   - `retrieve_all` method retrieves all nodes, including additional source nodes

   - Calls the recursive retrieval process and returns both retrieved and additional nodes


#### [`RetrieverQueryEngine`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/query_engine/retriever_query_engine.py)

The `index.as_retriever()` pattern creates a retriever from an index. The type of retriever it creates depends on the `retriever_mode` argument you pass to it. 

So, while `index.as_retriever()` is used to create a retriever from an index, a `RetrieverQueryEngine` uses a retriever to fetch relevant nodes and a `ResponseSynthesizer` to synthesize a response.

##### 🌿 Initialization
   - Takes a retriever, an optional response synthesizer, and node postprocessors

   - Creates a default response synthesizer if not provided
   
   - Sets up callback manager for the query engine and node postprocessors

##### 🔧 Customization

   - Can be initialized with various arguments using the `from_args` method

   - Allows customization of response mode, prompt templates, async usage, etc.

##### 🔄 Retrieval

   - Retrieves nodes using the provided retriever when `retrieve` or `aretrieve` is called

   - Applies node postprocessors to the retrieved nodes

   - Returns the processed nodes

##### 🔀 Retriever Swapping

   - Allows swapping the retriever using the `with_retriever` method

   - Creates a new RetrieverQueryEngine instance with the new retriever

##### 🧩 Node Postprocessing
   
   - Applies a list of node postprocessors to the retrieved nodes

   - Postprocessors can modify or filter the nodes based on the query bundle

##### 🎨 Response Synthesis

   - Synthesizes a response using the response synthesizer
   - Takes the query bundle, retrieved nodes, and additional source nodes as input
   - Generates a response based on the configured response mode and templates

##### ❓ Querying

   - Handles a query using the `_query` or `_aquery` method

   - Retrieves nodes, synthesizes a response, and returns the response

   - Triggers callback events for query start and end

##### 🏃 Async Support

   - Provides async versions of retrieval, synthesis, and querying methods

   - Allows for asynchronous processing of queries


In [30]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import RecursiveRetriever

parent_child_retriever = parent_child_index.as_retriever(
    response_mode="compact",
    similiarty_top_k=5,
    vector_store_query_mode="mmr", 
    vector_store_kwargs={"mmr_threshold": 0.42},
    )

retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": parent_child_retriever},
    node_dict=all_nodes_dict,
    verbose=True,
)

parent_child_query_engine = RetrieverQueryEngine.from_args(retriever_chunk, llm=Settings.llm)

parent_child_query_engine.update_prompts({'response_synthesizer:text_qa_template':HYPE_ANSWER_GEN_PROMPT_TEMPLATE})

### 🔧Setup Query Pipline for Parent Child Chunks

In [31]:
from utils import create_query_pipeline

from llama_index.core.query_pipeline import InputComponent

input_component = InputComponent()

parent_child_chain = [input_component, parent_child_query_engine]

parent_child_query_pipeline = create_query_pipeline(parent_child_chain)

In [32]:
parent_child_query_pipeline.run(input="How can I effectively build strength across multiple facets of real life without relying on complicated machines?")

> Running module 3cb61b58-ce2f-4eb0-9fa7-bc55219f6805 with input: 
input: How can I effectively build strength across multiple facets of real life without relying on complicated machines?

> Running module 2f356138-a1fd-409b-b232-78ee06c20690 with input: 
input: How can I effectively build strength across multiple facets of real life without relying on complicated machines?

Retrieving with query id None: How can I effectively build strength across multiple facets of real life without relying on complicated machines?
Retrieved node with id, entering: 5acad8f0-19f8-4e28-8ff7-f2c4a46e7d04
Retrieving with query id 5acad8f0-19f8-4e28-8ff7-f2c4a46e7d04: How can I effectively build strength across multiple facets of real life without relying on complicated machines?
Retrieved node with id, entering: 4a4c5d5f-c4f8-499d-a88f-d2a2811822bd
Retrieving with query id 4a4c5d5f-c4f8-499d-a88f-d2a2811822bd: How can I effectively build strength across multiple facets of real life without relying on com

Response(response="Listen up, you're not a machine, you're a human being with an incredible capacity to adapt and grow. You don't need complicated machines to build strength. Embrace the power of your own body and mind. You've got to be willing to learn and adapt, just like those who resist new tools or technologies. You've got to be willing to step out of your comfort zone and try new things. You've got to be willing to fail and learn from your mistakes. You've got to be willing to redesign yourself, just like Facebook or Twitter. You've got to be willing to accept that there are no permanent solutions in a dynamic system. You've got to be willing to make decisions that optimize for the long term, not just the short term. You've got to be willing to accept that tension is who you think you should be, and relaxation is who you are. You've got to be willing to learn the skills of decision-making. You've got to be willing to forge a new identity, just like Bruce Lee. You've got to be wil

In [33]:
parent_child_query_pipeline.run(input="How can I set rules and speak honestly without worrying about hurting someone's feelings?")

> Running module 3cb61b58-ce2f-4eb0-9fa7-bc55219f6805 with input: 
input: How can I set rules and speak honestly without worrying about hurting someone's feelings?

> Running module 2f356138-a1fd-409b-b232-78ee06c20690 with input: 
input: How can I set rules and speak honestly without worrying about hurting someone's feelings?

Retrieving with query id None: How can I set rules and speak honestly without worrying about hurting someone's feelings?
Retrieved node with id, entering: 69415430-15d7-43b3-a57c-d5b24e2bd2d5
Retrieving with query id 69415430-15d7-43b3-a57c-d5b24e2bd2d5: How can I set rules and speak honestly without worrying about hurting someone's feelings?
Retrieved node with id, entering: 0ace74d2-f90e-457b-98f3-a6435e2f1c03
Retrieving with query id 0ace74d2-f90e-457b-98f3-a6435e2f1c03: How can I set rules and speak honestly without worrying about hurting someone's feelings?


Response(response="Set your standards high and stick to them. Treat others as you wish to be treated. Be loyal, intelligent, and develop yourself fully. This is your foundation. You are in control. When it's time to set rules or speak honestly, do not hold back. Say what must be said. You are judged by your acts, not by how others feel about you. You must be honest. Be blunt. Be positive. Be confident. Be loving. Be charismatic. Be specific with praise. Be general with criticism. Do not criticize the person. Criticize the approach or the class of activities. You must be honest. You must be honest. You must be honest. Start now. You must be honest. You must be honest. You must be honest.", source_nodes=[NodeWithScore(node=IndexNode(id_='69415430-15d7-43b3-a57c-d5b24e2bd2d5', embedding=[-0.04498291015625, 0.04779052734375, 0.03448486328125, -0.026702880859375, 0.016937255859375, 0.01165771484375, 0.0214385986328125, -0.0242767333984375, 0.017974853515625, 0.0015649795532226562, -0.023834